In [1]:
import numpy as np
import cv2
import glob
import json
import dlib
from imutils import face_utils

In [11]:
def show_image(image, label):
    cv2.imshow(label, image)
    ans = cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return ans

In [2]:
participants = glob.glob('dataset/*')
print(participants[0])

dataset/0656f5fe35a54d1589e526a702f578b0


In [3]:
# Load the dataset
with open('data_cleaned.json') as json_file:
    data_all = json.load(json_file)
# Extract the keys in sorted order
keys_all = sorted(data_all)
# Convert python list to np array
keys_all = np.asarray(keys_all)

In [4]:
# Number of training examples to use(0-2758)
DATASET_SIZE = len(keys_all)
DEBUG = False

In [5]:
detector = dlib.get_frontal_face_detector()

In [23]:
faces = np.zeros((503, 224, 224, 3))
counter = 0

for i in range(DATASET_SIZE):
    key = keys_all[i]
    im = cv2.imread('dataset/' + key)
    if(im.shape == (720, 1280, 3)):
        rects = detector(im, 0)

        for rect in rects:
            # convert dlib's rectangle to a OpenCV-style bounding box
            # [i.e., (x, y, w, h)], then draw the face bounding box
            (x, y, w, h) = face_utils.rect_to_bb(rect)
            
            # Get rid of small faces
            if(w < 200 and h < 200):
                continue
                
            face = im[y:y+h, x:x+w]
            
            # Resize to match VGGFace requirements
            face = cv2.resize(face, (224, 224))
            faces[counter, :] = face
            counter += 1

#         cv2.imshow('image', im)
#         ans = cv2.waitKey(0)
#         cv2.destroyAllWindows()
#         if ans == ord('q'): break

In [27]:
print(counter)
faces = faces[:502, :]

502


In [28]:
print(faces.shape)

(502, 224, 224, 3)


In [31]:
from keras_vggface.vggface import VGGFace

Using TensorFlow backend.


In [35]:
model = VGGFace(model='resnet50', include_top=False, pooling='avg')

94699520/94694792 [==============================] - 212s 2us/step


In [36]:
features = model.predict(faces)

In [37]:
print(features.shape)

(502, 2048)
